# DAT 108 – Lecture 41 Review

In [ ]:
import numpy as np
from datascience import *

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

## Movies Dataset

In [ ]:
movies = Table.read_table('movies_fakedata.csv')
movies.show(5)

In [ ]:
# Average revenue of all movies
average_revenue = movies.column('Revenue').mean()
average_revenue

In [ ]:
# Number of movies per genre
movies_per_genre = movies.group('Genre')
movies_per_genre

In [ ]:
# Average revenue per genre
average_revenue_per_genre = movies.group('Genre', np.mean)
average_revenue_per_genre

## Students and Grades

In [ ]:
students = Table.read_table('students_fakedata.csv')
grades = Table.read_table('grades_fakedata.csv')
combined_table = students.join('Student_ID', grades, 'Student_ID')
combined_table.show(5)

In [ ]:
def standard_units(x):
    return (x - np.mean(x)) / np.std(x)

In [ ]:
def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)

correlation(combined_table, 'Time tutored', 'Exam_Score')

## Survey Dataset

In [ ]:
survey = Table.read_table('survey_fakedata.csv')
survey.show(5)

In [ ]:
# Number of students and non-students by drink
survey.pivot('Preferred_Drink', 'Student')

In [ ]:
# Average study hours by sex and drink for students
survey.where('Student', 1).pivot('Preferred_Drink', 'Sex', values='Study_Hours', collect=np.mean)

## Sampling

In [ ]:
population = Table.read_table('population_fakedata.csv')
# Random sample of 10
population.sample(10)

In [ ]:
# Specific indices
population.take([0, 9, 99])

## Hypothesis Testing

In [ ]:
scores = Table.read_table('scores_fakedata.csv')
scores.group('Method', np.mean)

In [ ]:
def difference_in_means(table):
    group_means = table.group('Method', np.mean)
    return group_means.where('Method', 'B').column('Score mean').item(0) - \
           group_means.where('Method', 'A').column('Score mean').item(0)

difference_in_means(scores)

In [ ]:
def permuted_test_stat():
    shuffled = scores.with_column('Shuffled', scores.sample(with_replacement=False).column('Method'))
    shuffled_table = shuffled.select('Shuffled', 'Score').relabeled('Shuffled', 'Method')
    return difference_in_means(shuffled_table)

simulated_stats = make_array()
repetitions = 1000
for i in np.arange(repetitions):
    simulated_stats = np.append(simulated_stats, permuted_test_stat())

In [ ]:
type(simulated_stats), simulated_stats.shape

In [ ]:
sim_table = Table().with_column('Simulated Statistic', simulated_stats)
sim_table.hist('Simulated Statistic')
#observed_stat = difference_in_means(scores)
#print("Observed statistic:", observed_stat)

In [ ]:
p_value = np.count_nonzero(simulated_stats >= observed_stat) / repetitions
p_value